# Spectral Event Analysis Tutorial

This tutorial is a hands-on introduction to using the [SpectralEvents toolbox](https://github.com/jonescompneurolab/SpectralEvents). Here, we'll load and analyze data as described in [Shin et al. (2017)](https://doi.org/10.7554/eLife.29086) that is distributed with the toolbox.

First, we'll import some dependencies. Note that `seaborn` is not required, but is added to improve plot styling.

In [ ]:
%matplotlib inline

import sys
import os.path as op
from glob import glob

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

Now let's import the SpectralEvents toolbox module. You might need to modify the path to this module so that your Python interpreter knows where to find it.

In [ ]:
# set path to SpectralEvents if necessary
#sys.path.append('/home/ryan/SpectralEvents')
import spectralevents as se

XXX find data_dir automatically

In [ ]:
# dataset parameters
data_dir = 'data'  # relative path to data directory
subj_ids = [str(id) for id in range(1, 10 + 1)]  # subject IDs 1-10

n_subjs = len(subj_ids)  # number of subjects
n_trials = 200           # number of trials per subject
n_times = 600            # number of time samples per trial
samp_freq = 600          # sampling rate (Hz)

In [ ]:
# load data
hit_trials = list()
data = list()
for id_idx, id in enumerate(subj_ids):
    fname = op.join(data_dir, 'prestim_humandetection_600hzMEG_subject' + id + '.mat')
    raw_data = loadmat(fname)
    hit_trials.append(np.nonzero(raw_data['YorN'].squeeze()))  # indices of hit trials
    data.append(raw_data['prestim_raw_yes_no'])  # MEG time series (trials x samples)

Once you've loaded the data, make sure you understand how it's formatted. Note that there are 10 subjects, each with 100 "hit" trials and 100 "miss" trials. How might you go about selecting only the "hit" trials from the 1st subject?

In [ ]:
# investigate data structure
data[0].shape  # trials x samples for Subj 1

We've loaded the data and are now ready to try detecting some Spectral Events. Three important parameters will govern the outcome of your results: the frequencies `freqs` over which you will calculate your time-frequency response, the time values `times` at which your signal was sampled relative to each epoch (i.e, trial), and the bounds of the frequency band `event_band` in which you will look for Spectral Events.

In [ ]:
# set time-frequency response (TFR) parameters
freqs = list(range(1, 60 + 1))   # TFR: fequency values (Hz) over which to calculate
times = np.arange(n_times) / samp_freq  # seconds

# set spectral event analysis parameters
event_band = [15, 29]  # frequency range (Hz) of spectral events (beta band)

In [ ]:
# calculate time-frequency response (TFR)
tfrs = np.zeros((n_subjs, n_trials, len(freqs), n_times))

for subj_idx, subj_data in enumerate(data):

    # calculate TFR using the Morlet wavelet method
    tfr = se.tfr(subj_data, freqs, samp_freq)
    tfrs[subj_idx, :, :, :] = tfr

In [ ]:
# run spectral event analysis per subject
spec_events_all = list()

for subj_idx, subj_data in enumerate(data):
    tfr = tfrs[subj_idx]

    # find local maxima in TFR
    spec_events = se.find_events(tfr=tfr, times=times, freqs=freqs,
                                 event_band=event_band, threshold_FOM=6.)
    spec_events_all.append(spec_events)


In [ ]:
fig = se.plot_avg_spectrogram(spec_events=spec_events,
                              tfr=tfr, times=times, freqs=freqs,
                              event_band=event_band, timeseries=subj_data,
                              example_epochs=[43, 6, 99], vlim=None)  # try vlim=[0, 1.0e-17]